<a href="https://colab.research.google.com/github/KediyaVis/Regression/blob/main/REG02_NB01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing Turi Create functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in Turi Create functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up Turi Create

In [3]:
import pandas as pd

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [4]:
from google.colab import files
uploaded = files.upload()

Saving kc_house_train_data.csv to kc_house_train_data.csv
Saving kc_house_test_data.csv to kc_house_test_data.csv


# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let Turi Create pick a random seed for you).  

In [6]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

In [7]:
train_data = pd.read_csv('kc_house_train_data.csv').astype(dtype_dict)
test_data = pd.read_csv('kc_house_test_data.csv').astype(dtype_dict)

In [9]:
train_data.columns

Index([u'id', u'date', u'price', u'bedrooms', u'bathrooms', u'sqft_living',
       u'sqft_lot', u'floors', u'waterfront', u'view', u'condition', u'grade',
       u'sqft_above', u'sqft_basement', u'yr_built', u'yr_renovated',
       u'zipcode', u'lat', u'long', u'sqft_living15', u'sqft_lot15'],
      dtype='object')

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [8]:
 from sklearn.linear_model import LinearRegression

In [10]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model  = LinearRegression().fit(train_data[example_features], train_data['price'])

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [12]:
example_model.coef_

array([   315.40669062, -65081.88711588,   6942.16598637])

# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing Turi Create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [13]:
example_predictions = example_model.predict(train_data[example_features])
print example_predictions[0] # should be 271789.505878

271789.26537996973


# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [26]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    pred = model.predict(data)
    # Then compute the residuals/errors
    res=pred-outcome
    # Then square and add them up
    RSS=(res*res).sum()
    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [27]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data[example_features], test_data['price'])
print rss_example_train # should be 2.7376153833e+14

273761940583133.75


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [28]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [29]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [34]:
# create the remaining 3 features in both TEST and TRAIN data
import numpy as np
train_data['bed_bath_rooms'] = train_data['bedrooms']*train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms']*test_data['bathrooms']

train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))

train_data['lat_plus_long'] = train_data['lat']+train_data['long']
test_data['lat_plus_long'] = test_data['lat']+test_data['long']


* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [35]:
test_data[['bedrooms_squared','bed_bath_rooms','log_sqft_living','lat_plus_long']].mean()

bedrooms_squared    12.446678
bed_bath_rooms       7.503902
log_sqft_living      7.550275
lat_plus_long      -74.653334
dtype: float64

# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [36]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using turicreate.linear_regression.create() and look at the value of the weights/coefficients:

In [43]:
# Learn the three models: (don't forget to set validation_set = None)
model1  = LinearRegression().fit(train_data[model_1_features], train_data['price'])
model2  = LinearRegression().fit(train_data[model_2_features], train_data['price'])
model3  = LinearRegression().fit(train_data[model_3_features], train_data['price'])
print(model1.coef_,model2.coef_,model3.coef_)


(array([ 3.12258646e+02, -5.95865332e+04,  1.57067421e+04,  6.58619264e+05,
       -3.09374351e+05]), array([ 3.06610053e+02, -1.13446368e+05, -7.14613083e+04,  6.54844630e+05,
       -2.94298969e+05,  2.55796520e+04]), array([ 5.29422820e+02,  3.45142296e+04,  6.70607813e+04,  5.34085611e+05,
       -4.06750711e+05, -8.57050439e+03, -6.78858667e+03, -5.61831484e+05,
        1.27334900e+05]))


In [48]:
# Examine/extract each model's coefficients:
df1=pd.DataFrame(model1.coef_,model_1_features).rename(columns={0:'model1'})
df2=pd.DataFrame(model2.coef_,model_2_features).rename(columns={0:'model2'})
df3=pd.DataFrame(model3.coef_,model_3_features).rename(columns={0:'model3'})
print(df1,df2,df3)

(                    model1
sqft_living     312.258646
bedrooms     -59586.533154
bathrooms     15706.742083
lat          658619.263931
long        -309374.351268,                        model2
sqft_living        306.610053
bedrooms       -113446.368070
bathrooms       -71461.308293
lat             654844.629503
long           -294298.969138
bed_bath_rooms   25579.652001,                          model3
sqft_living          529.422820
bedrooms           34514.229578
bathrooms          67060.781319
lat               534085.610867
long             -406750.710861
bed_bath_rooms     -8570.504395
bedrooms_squared   -6788.586670
log_sqft_living  -561831.484076
lat_plus_long     127334.900006)


**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [52]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
R1=get_residual_sum_of_squares(model1, train_data[model_1_features], train_data['price'])
R2=get_residual_sum_of_squares(model2, train_data[model_2_features], train_data['price'])
R3=get_residual_sum_of_squares(model3, train_data[model_3_features], train_data['price'])
print(R1,R2,R3)
min([R1,R2,R3])

(967879963049545.8, 958419635074068.8, 903436455050478.0)


903436455050478.0

**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [54]:
# Compute the RSS on TESTING data for each of the three models and record the values:
R1=get_residual_sum_of_squares(model1, test_data[model_1_features], test_data['price'])
R2=get_residual_sum_of_squares(model2, test_data[model_2_features], test_data['price'])
R3=get_residual_sum_of_squares(model3, test_data[model_3_features], test_data['price'])
print(R1,R2,R3)
min([R1,R2,R3])

(225500469795489.56, 223377462976467.16, 259236319207179.3)


223377462976467.16

**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.